In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import pickle

import warnings
warnings.filterwarnings('ignore')

# Preprocessing and modeling

In [4]:
joined = pd.read_csv('joined data.csv')
joined.head()

product_id            brand       mpn  \
0  01DPGV4YRP3Z8J85DASGZ1Y99W            Frame  LWAX0056   
1  01DPGV4YRP3Z8J85DASGZ1Y99W            Frame  LWAX0056   
2  01DSE8Z2ZDAZKZ2SKCS1E3B3HK  Banana Republic    491075   
3  01DSE8Z2ZDAZKZ2SKCS1E3B3HK  Banana Republic    491075   
4  01E2C3YN4KQ36A0REWZJ89ZN73   FREDA SALVADOR   5229129   

             product_full_name  \
0    Les Second - Medium--NOIR   
1    Les Second - Medium--NOIR   
2  Madison 12-Hour Loafer Pump   
3  Madison 12-Hour Loafer Pump   
4                   Ace Bootie   

                                         description brand_category  \
0  Minimal, Modern Styling Meets Refined Luxury I...    Accessories   
1  Minimal, Modern Styling Meets Refined Luxury I...    Accessories   
2  Everything you love about our original Madison...        Unknown   
3  Everything you love about our original Madison...        Unknown   
4  Edgy style and expert craftsmanship combine on...        Unknown   

                         created_at                        updated_at  \
0     2019-10-06 15:31:31.730524+00     2019-12-19 20:40:30.786144+00   
1  2019-10-06 15:31:31.730000+00:00  2020-04-06 23:19:53.216000+00:00   
2  2019-11-11 22:22:21.664000+00:00  2020-03-25 23:24:44.823000+00:00   
3     2019-11-11 22:22:21.664425+00     2019-12-19 20:40:30.786144+00   
4  2020-03-01 22:37:32.169000+00:00  2020-04-15 21:46:03.512000+00:00   

                         deleted_at  \
0                               NaN   
1  2020-04-06 23:19:53.216000+00:00   
2  2020-03-23 21:06:15.953000+00:00   
3                               NaN   
4  2020-03-18 23:00:31.558000+00:00   

                                 brand_canonical_url  \
0  https://frame-store.com/products/les-second-me...   
1  https://frame-store.com/products/les-second-me...   
2  https://bananarepublic.gap.com/browse/product....   
3  https://bananarepublic.gap.com/browse/product....   
4  https://shop.nordstrom.com/s/freda-salvador-ac...   

                                             details labels  bc_product_id  \
0                                                NaN     {}            NaN   
1                                                NaN     []          185.0   
2  Everything you love about our original Madison...     []          431.0   
3  Everything you love about our original Madison...     {}            NaN   
4  True to size.\n2 1/4" (57mm) heel (size 8.5)\n...     []         1051.0   

                 product_id-2            product_color_id attribute_name  \
0  01DPGV4YRP3Z8J85DASGZ1Y99W  01DPGVGBK6YGNYGNF2S6FSH02T          style   
1  01DPGV4YRP3Z8J85DASGZ1Y99W  01DPGVGBK6YGNYGNF2S6FSH02T          style   
2  01DSE8Z2ZDAZKZ2SKCS1E3B3HK  01DSE8ZG8Y3FR8KWE2TY1QDWBF     shoe_width   
3  01DSE8Z2ZDAZKZ2SKCS1E3B3HK  01DSE8ZG8Y3FR8KWE2TY1QDWBF     shoe_width   
4  01E2C3YN4KQ36A0REWZJ89ZN73  01E2C3YN56ZCJ8TN45V3EC8CPS  Primary Color   

  attribute_value          file  
0          Casual  initial_tags  
1          Casual  initial_tags  
2          Medium  initial_tags  
3          Medium  initial_tags  
4          Blacks  initial_tags

In [5]:
joined.shape

(159013, 18)

In [6]:
joined[joined.attribute_name == 'pattern'].attribute_value.value_counts()

animal              436
logo                253
floral              209
plaid               198
stripevertical      129
geometric           127
stripehorizontal    102
graphic              94
dots                 64
abstract             57
stripe               43
colorblock           42
tiedye               33
tropical             24
camouflage           22
multiprint           19
pinstripe            18
paisley              17
houndstooth          12
monogram              3
Name: attribute_value, dtype: int64

In [7]:
joined[joined.attribute_name == 'print'].attribute_value.value_counts()

none    4
Name: attribute_value, dtype: int64

In [8]:
pattern = joined[joined.attribute_name == 'pattern']

In [9]:
## Remove unrelevant columns and then remove duplicates

pattern = pattern.drop(['mpn', 'created_at', 'updated_at', 'deleted_at', 'brand_canonical_url', 'labels', 
                        'bc_product_id', 'product_id-2', 'file'], axis = 1)
pattern = pattern.drop_duplicates()

In [10]:
pattern.head()

product_id            brand  \
10746  01DTJ8DQ5SQYBS5HX6TDH2BSJP   Hunting Season   
10764  01DTJ8DQ5SQYBS5HX6TDH2BSJP   Hunting Season   
10780  01DTATGG2XJ7QTQC6A4PHQRYW5           Alexis   
10792  01DTATGSMSV7FJYTPVAYEHPD8Z        Derek Lam   
10827  01DVCRRBK29DWT82DRBVVY544X  Mother of Pearl   

                                       product_full_name  \
10746                Leather-Paneled Raffia Shoulder Bag   
10764                Leather-Paneled Raffia Shoulder Bag   
10780                      Calixte Pussy-Bow Chiffon Top   
10792  Belted lace-trimmed printed silk-crepe maxi dress   
10827                                             #NAME?   

                                             description  \
10746  Handwoven by local artisans, Hunting Season's ...   
10764  Handwoven by local artisans, Hunting Season's ...   
10780  Alexis' 'Calixte' top has a vintage '60s feel ...   
10792  Multicolored silk-crepe Slips on 100% silk; li...   
10827  Black and white Lyocell Partially concealed sn...   

                  brand_category  \
10746   women:BAGS:SHOULDER_BAGS   
10764   women:BAGS:SHOULDER_BAGS   
10780        women:CLOTHING:TOPS   
10792  Clothing / Dresses / Maxi   
10827  Clothing / Tops / Blouses   

                                                 details  \
10746  Drawstring top\nComposition: leather, raffia (...   
10764  Drawstring top\nComposition: leather, raffia (...   
10780  Tie fastenings at neck\nComposition: 100% poly...   
10792  Fits true to size, take your normal size \nTho...   
10827  Fits true to size, take your normal size \nCut...   

                 product_color_id attribute_name attribute_value  
10746  01DTJ8DS7T1QXA88MF9T17ZKS9        pattern      colorblock  
10764  01DTJ8DQ64FXG2PTNDZX5QHX4D        pattern      colorblock  
10780  01DTATGG356E9CF3V4TWB705BC        pattern            dots  
10792  01DTATGSN4RWS0SJYJJCWR8PA6        pattern       geometric  
10827  01DVCRRBKF3E3MTGQR5NYADG5N        pattern       geometric

In [11]:
pattern.shape

(1612, 9)

In [12]:
pattern.isnull().sum()

product_id             0
brand                  0
product_full_name      0
description          228
brand_category        42
details               82
product_color_id       0
attribute_name         0
attribute_value        0
dtype: int64

In [13]:
#replace null values with UNKNOWN_TOKEN
pattern['description'] = pattern['description'].fillna('UNKNOWN_TOKEN')
pattern['details'] = pattern['details'].fillna('UNKNOWN_TOKEN')
pattern['brand_category'] = pattern['brand_category'].fillna('UNKNOWN_TOKEN')

In [14]:
pattern.isnull().sum()

product_id           0
brand                0
product_full_name    0
description          0
brand_category       0
details              0
product_color_id     0
attribute_name       0
attribute_value      0
dtype: int64

In [15]:
pattern['details'] = pattern['details'].str.replace("\n", "")

### Preprocessing

In [16]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    pattern[col] = pattern[col].apply(removePunctuation)
    pattern[col] = pattern[col].apply(removeStopwords)
    pattern[col] = pattern[col].apply(lemmatize)

# generate dummy variables for all different attribute value levels
dummies = pd.get_dummies(pattern['attribute_value'])
pattern = pd.concat([pattern, dummies], axis = 1)
pattern.reset_index(inplace = True)
pattern = pattern.drop('index', axis = 1)

### Models

In [17]:
pattern["input_doc"] = pattern.brand + " " + pattern.product_full_name + " " + pattern.description + " " + pattern.details 

In [18]:
pattern.head()

product_id           brand  \
0  01DTJ8DQ5SQYBS5HX6TDH2BSJP  hunting season   
1  01DTJ8DQ5SQYBS5HX6TDH2BSJP  hunting season   
2  01DTATGG2XJ7QTQC6A4PHQRYW5          alexis   
3  01DTATGSMSV7FJYTPVAYEHPD8Z       derek lam   
4  01DVCRRBK29DWT82DRBVVY544X    mother pearl   

                                   product_full_name  \
0                leather-paneled raffia shoulder bag   
1                leather-paneled raffia shoulder bag   
2                      calixte pussy-bow chiffon top   
3  belted lace-trimmed printed silk-crepe maxi dress   
4                                               name   

                                         description  \
0  handwoven local artisan hunting season 's shou...   
1  handwoven local artisan hunting season 's shou...   
2  alexis 'calixte top vintage '60s feel somehow ...   
3  multicolored silk-crepe slip 100 silk lining 1...   
4  black white lyocell partially concealed snap f...   

              brand_category  \
0   women:BAGS:SHOULDER_BAGS   
1   women:BAGS:SHOULDER_BAGS   
2        women:CLOTHING:TOPS   
3  Clothing / Dresses / Maxi   
4  Clothing / Tops / Blouses   

                                             details  \
0  drawstring topcomposition leather raffia plata...   
1  drawstring topcomposition leather raffia plata...   
2  tie fastening neckcomposition 100 polyesterdry...   
3  fit true size take normal size petite frame ma...   
4  fit true size take normal size cut relaxed fit...   

             product_color_id attribute_name attribute_value  abstract  ...  \
0  01DTJ8DS7T1QXA88MF9T17ZKS9        pattern      colorblock         0  ...   
1  01DTJ8DQ64FXG2PTNDZX5QHX4D        pattern      colorblock         0  ...   
2  01DTATGG356E9CF3V4TWB705BC        pattern            dots         0  ...   
3  01DTATGSN4RWS0SJYJJCWR8PA6        pattern       geometric         0  ...   
4  01DVCRRBKF3E3MTGQR5NYADG5N        pattern       geometric         0  ...   

   multiprint  paisley  pinstripe  plaid  stripe  stripehorizontal  \
0           0        0          0      0       0                 0   
1           0        0          0      0       0                 0   
2           0        0          0      0       0                 0   
3           0        0          0      0       0                 0   
4           0        0          0      0       0                 0   

   stripevertical  tiedye  tropical  \
0               0       0         0   
1               0       0         0   
2               0       0         0   
3               0       0         0   
4               0       0         0   

                                           input_doc  
0  hunting season leather-paneled raffia shoulder...  
1  hunting season leather-paneled raffia shoulder...  
2  alexis calixte pussy-bow chiffon top alexis 'c...  
3  derek lam belted lace-trimmed printed silk-cre...  
4  mother pearl name black white lyocell partiall...  

[5 rows x 30 columns]

### Demo modeling on 'abstract'

In [19]:
X = pattern.iloc[:, -1].values
y = pattern.loc[:, "abstract"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

In [20]:
docs = list(X_train)
labels = y_train

In [21]:
# set the vocabulary size to some number that represents the total number of unique words in your vocabulary
vocab_size = 5000

In [22]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
tokenizer.word_index

{'size': 1,
 'true': 2,
 '2': 3,
 'fit': 4,
 '5': 5,
 '1': 6,
 '4': 7,
 '8': 8,
 "'s": 9,
 'length': 10,
 'sleeve': 11,
 's': 12,
 '6': 13,
 '10': 14,
 'leather': 15,
 'cotton': 16,
 'style': 17,
 'button': 18,
 'print': 19,
 'waist': 20,
 '0': 21,
 'model': 22,
 'l': 23,
 'wash': 24,
 'small': 25,
 'm': 26,
 'xs': 27,
 'top': 28,
 '100': 29,
 '12': 30,
 'silk': 31,
 'dress': 32,
 'x': 33,
 'dry': 34,
 'wearing': 35,
 'detail': 36,
 'unknown': 37,
 'token': 38,
 '14': 39,
 'body': 40,
 'bust': 41,
 'front': 42,
 'high': 43,
 '24': 44,
 'shirt': 45,
 'xl': 46,
 'hip': 47,
 'rail': 48,
 'classic': 49,
 'imported': 50,
 'stripe': 51,
 'height': 52,
 'made': 53,
 'strap': 54,
 'measurement': 55,
 'nike': 56,
 '9': 57,
 'soft': 58,
 'long': 59,
 'tory': 60,
 'shoe': 61,
 'hand': 62,
 'cut': 63,
 'neck': 64,
 'rayon': 65,
 'one': 66,
 '00': 67,
 'back': 68,
 'cold': 69,
 'pocket': 70,
 'care': 71,
 'recommend': 72,
 'measured': 73,
 '32': 74,
 'bleach': 75,
 'burch': 76,
 'lining': 77,
 'lin

In [23]:
from keras.preprocessing.text import text_to_word_sequence

In [24]:
from typing import List
def integer_encode_documents(docs: List[str], tokenizer: Tokenizer)-> List[List[int]]:
    documents = []
    for d in docs:
        doc_integers = []
        for i in text_to_word_sequence(d):
            doc_integers.append(tokenizer.word_index[i])
        documents.append(doc_integers)
    return documents

In [25]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [26]:
# integer encode the documents
encoded_docs = integer_encode_documents(docs, tokenizer)

from pprint import pprint
pprint(encoded_docs)

[[60,
  76,
  679,
  2062,
  79,
  15,
  184,
  184,
  226,
  262,
  9,
  92,
  313,
  1252,
  1720,
  135,
  2062,
  466,
  186,
  60,
  9,
  266,
  208,
  111,
  79,
  27,
  252,
  12,
  193,
  26,
  90,
  23,
  369,
  46,
  281,
  933,
  934,
  1017,
  6,
  184,
  557,
  6,
  6,
  7,
  370,
  2848],
 [56,
  49,
  1434,
  680,
  381,
  86,
  270,
  221,
  198,
  227,
  2063,
  6,
  558,
  411,
  1435,
  161,
  381,
  178,
  116,
  42,
  56,
  2849,
  1,
  2064,
  2850,
  1,
  2851,
  1253,
  2852,
  66,
  935,
  2064,
  314,
  2853,
  2854,
  4,
  816,
  2855,
  1,
  4,
  641,
  50,
  4,
  2,
  1,
  93,
  97,
  1,
  187,
  147],
 [56,
  124,
  303,
  1436,
  2065,
  86,
  681,
  106,
  315,
  864,
  139,
  1721,
  682,
  2066,
  1104,
  1254,
  86,
  1018,
  443,
  304,
  1437,
  683,
  2,
  1],
 [684,
  642,
  685,
  643,
  150,
  510,
  130,
  123,
  64,
  130,
  2067,
  150,
  510,
  175,
  16,
  241,
  2856,
  1105,
  2,
  1,
  27,
  21,
  12,
  3,
  7,
  26,
  13,
  8,
  23,
  1

  11,
  1031,
  1283,
  104,
  452,
  185,
  2111,
  725,
  2112,
  68,
  172,
  2,
  1,
  27,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39,
  149,
  10,
  1,
  142,
  42,
  18,
  939,
  64,
  2113,
  2907,
  769,
  11,
  66,
  18,
  828,
  400,
  419,
  189],
 [48,
  497,
  37,
  38,
  36,
  58,
  175,
  88,
  11,
  123,
  64,
  180,
  28,
  170,
  1284,
  1135,
  313,
  49,
  161,
  95,
  655,
  1136,
  19,
  264,
  433,
  234,
  78,
  168,
  449,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25,
  22,
  9,
  55,
  52,
  5,
  98,
  41,
  74,
  20,
  44,
  47,
  90],
 [656,
  2114,
  293,
  28,
  1285,
  1286,
  1468,
  1760,
  2115,
  323,
  302,
  19,
  318,
  199,
  45,
  248,
  830,
  196,
  16,
  2,
  1,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  39,
  102,
  6,
  3,
  10,
  1,
  142,
  42,
  18,
  1029,
  1469,
  2908,
  516,
  2909,
  16,
  3,
  1137,
  82,
  249,
  24,
  15

  138,
  53,
  2966,
  2076,
  1500,
  167,
  1108,
  2967,
  1781,
  2968,
  2,
  1],
 [48,
  497,
  37,
  38,
  36,
  58,
  175,
  88,
  11,
  123,
  64,
  180,
  28,
  170,
  1284,
  1135,
  313,
  49,
  161,
  95,
  780,
  1136,
  19,
  264,
  433,
  234,
  78,
  168,
  449,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25,
  22,
  9,
  55,
  52,
  5,
  98,
  41,
  74,
  20,
  44,
  47,
  90],
 [457,
  2155,
  2155,
  627,
  53,
  58,
  964,
  100,
  355,
  66,
  413,
  1501,
  2156,
  354,
  151,
  99,
  127,
  123,
  64,
  953,
  1782,
  184,
  721,
  1151,
  544,
  1502,
  1503,
  125,
  698,
  19,
  22,
  889,
  458,
  5,
  8,
  344,
  109,
  1,
  12,
  40,
  29,
  964,
  77,
  29,
  167,
  36,
  29,
  602,
  18,
  1152,
  545,
  627,
  125,
  1783,
  24,
  307,
  890,
  1504,
  1505,
  546,
  158,
  1506,
  127,
  547,
  839,
  875,
  34,
  82,
  501,
  502,
  1153],
 [439,
  1281,
  150,
  2157,
  129,
  2158,


  366,
  337,
  104,
  3021,
  203,
  179,
  1485,
  311,
  232,
  289,
  17,
  1173,
  135,
  979,
  121,
  36,
  1811,
  68,
  294,
  1027,
  107,
  70,
  408,
  741,
  525,
  19,
  203,
  222,
  535,
  254,
  449,
  281,
  78,
  50,
  71,
  249,
  24,
  69,
  158,
  34,
  127,
  1812,
  2183,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  126,
  73,
  25,
  22,
  35,
  1,
  12,
  22,
  9,
  55,
  52,
  5,
  57,
  41,
  157,
  6,
  3,
  20,
  133,
  6,
  3,
  47,
  169],
 [283,
  3022,
  300,
  387,
  19,
  129,
  387,
  19,
  93,
  472,
  570,
  1813,
  510,
  51,
  456,
  980,
  129,
  202,
  742,
  1518,
  343,
  894,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 [902,
  2184,
  345,
  19,
  201,
  250,
  352,
  485,
  942,
  1814,
  186,
  106,
  902,
  9,
  981,
  418,
  2185,
  326,
  350,
  9,
  2186,
  61,
  494,
  690,
  9,
  2187,
  352,
  248,
  882,
  345,
  180,
  201,
  250,
  290,
  647,
  113,
  99,
  1530,
  15,
 

  8,
  14,
  23,
  30,
  39,
  46,
  102,
  153],
 [219,
  415,
  1270,
  142,
  277,
  15,
  239,
  763,
  591,
  213,
  289,
  1734,
  321,
  492,
  277,
  15,
  239,
  1271,
  222,
  1110,
  17,
  247,
  136,
  33,
  263,
  91,
  143,
  33,
  13,
  145,
  162,
  183,
  223,
  820,
  14,
  136,
  33,
  98,
  91,
  143,
  33,
  514,
  5,
  91,
  471,
  8,
  54,
  146,
  3,
  6,
  188],
 [48,
  497,
  37,
  38,
  36,
  58,
  175,
  88,
  11,
  123,
  64,
  180,
  282,
  28,
  203,
  282,
  4,
  343,
  109,
  118,
  3080,
  131,
  1302,
  19,
  264,
  433,
  234,
  78,
  168,
  449,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25,
  22,
  35,
  1,
  12,
  22,
  9,
  55,
  52,
  5,
  98,
  5,
  41,
  90,
  20,
  44,
  47,
  229,
  5],
 [60,
  76,
  679,
  616,
  105,
  9,
  589,
  124,
  113,
  1255,
  60,
  271,
  759,
  274,
  105,
  360,
  346,
  54,
  256,
  148,
  4,
  2,
  1],
 [60,
  76,
  1193,
  51,
  584,
  1159

...

  136,
  33,
  13,
  143,
  33,
  7,
  145,
  162,
  183,
  25,
  30,
  91,
  54,
  146,
  112,
  91,
  359,
  54,
  146,
  6,
  5,
  188,
  571,
  496,
  294,
  2502,
  1091,
  87,
  174,
  707,
  156,
  2503,
  2504],
 [219,
  4536,
  1652,
  1988,
  466,
  184,
  1175,
  15,
  184,
  373,
  499,
  451,
  783,
  1103,
  1988,
  1161,
  921,
  184,
  308,
  932,
  20,
  267,
  186,
  37,
  38],
 [48,
  497,
  37,
  38,
  36,
  58,
  175,
  88,
  11,
  123,
  64,
  180,
  28,
  234,
  78,
  168,
  449,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  4,
  72,
  101,
  2,
  1,
  40,
  10,
  44,
  6,
  3,
  73,
  25],
 [60,
  76,
  811,
  15,
  508,
  110,
  62,
  937,
  1618,
  722,
  566,
  170,
  1226,
  238,
  1090,
  821,
  1227,
  174,
  110,
  556,
  411,
  79,
  1383,
  8,
  136,
  33,
  13,
  143,
  33,
  6,
  91,
  145,
  162,
  183,
  25,
  112,
  44,
  174,
  54,
  146,
  6,
  21,
  188,
  28,
  84,
  669,
  341,
  232,
  15,
  174,
  707,
  156,
  1384,
  572,
  750],
 [101

  2173,
  156,
  53,
  1945,
  1,
  700,
  1,
  4,
  641,
  4,
  25,
  1,
  93,
  368,
  1,
  1235,
  1403,
  147],
 [283,
  4579,
  2056,
  123,
  64,
  31,
  28,
  37,
  38,
  2,
  1,
  80,
  67,
  27,
  21,
  3,
  12,
  7,
  13,
  26,
  8,
  14,
  23,
  30,
  46,
  39],
 [216,
  680,
  609,
  11,
  667,
  520,
  454,
  64,
  1387,
  10,
  609,
  11,
  36,
  1770,
  667,
  63,
  1060,
  680,
  257,
  2,
  1],
 [1784,
  1785,
  1786,
  4580,
  31,
  4581,
  129,
  37,
  38,
  273,
  440,
  235,
  782,
  840,
  164,
  350,
  9,
  968,
  257,
  141,
  151,
  115,
  1513,
  662,
  4582,
  19,
  226,
  28,
  125,
  20,
  4583,
  4584,
  29,
  400,
  583,
  35,
  298,
  1,
  969,
  1155,
  17,
  4,
  2,
  165,
  160,
  155,
  97,
  1],
 [60,
  76,
  1688,
  1421,
  365,
  239,
  49,
  239,
  53,
  575,
  365,
  401,
  111,
  79,
  477,
  1421,
  657,
  15,
  264,
  787,
  1992,
  159,
  485,
  4585,
  469,
  136,
  33,
  30,
  143,
  33,
  5,
  145,
  162,
  183,
  338,
  223,
  8,
  54,
 

  131,
  161,
  920,
  795,
  528,
  496,
  182,
  42,
  29,
  920,
  34,
  82,
  50,
  4,
  2,
  1,
  93,
  97,
  1,
  63,
  108,
  926,
  423,
  529,
  253,
  371,
  372,
  210,
  35,
  1098,
  8],
 [48,
  258,
  37,
  38,
  36,
  49,
  48,
  59,
  11,
  176,
  18,
  85,
  45,
  53,
  132,
  204,
  58,
  65,
  96,
  118,
  185,
  287,
  284,
  17,
  297,
  320,
  172,
  49,
  347,
  141,
  348,
  106,
  114,
  179,
  66,
  107,
  70,
  29,
  65,
  50,
  71,
  62,
  24,
  69,
  83,
  34,
  75,
  75,
  4,
  2,
  1,
  166,
  4,
  367,
  394,
  4,
  72,
  147,
  42,
  40,
  10,
  134,
  98,
  8,
  73,
  25,
  22,
  35,
  1,
  12,
  22,
  9,
  55,
  52,
  5,
  57,
  41,
  157,
  6,
  3,
  20,
  133,
  6,
  3,
  47,
  169],
 [48,
  2182,
  37,
  38,
  36,
  137,
  59,
  11,
  161,
  846,
  524,
  18,
  85,
  176,
  28,
  118,
  650,
  1171,
  36,
  87,
  301,
  903,
  340,
  819,
  154,
  567,
  823,
  36,
  292,
  17,
  135,
  58,
  65,
  96,
  1759,
  284,
  1320,
  445,
  740,
  1810,
 

  164,
  786,
  894,
  493,
  979,
  468,
  29,
  167,
  249,
  24,
  69,
  53,
  206,
  17,
  405,
  1485,
  179,
  358,
  32,
  200,
  1630,
  588,
  315,
  4684,
  2027,
  4,
  405,
  190,
  181,
  186,
  215,
  2,
  1,
  452,
  4685,
  32,
  443,
  998,
  1106,
  4686,
  200,
  509,
  22,
  5,
  14,
  35,
  1,
  33,
  55,
  33,
  41,
  665,
  1635,
  10,
  133,
  899,
  443,
  41,
  858,
  858,
  10,
  134,
  356,
  1429,
  41,
  956,
  4687,
  10,
  134,
  4688,
  1430,
  41,
  1278,
  2036,
  10,
  149,
  469,
  29,
  4689,
  206],
 [1990,
  4690,
  323,
  302,
  152,
  32,
  927,
  358,
  789,
  63,
  425,
  385,
  4691,
  323,
  302,
  19,
  408,
  1663,
  376,
  1245,
  2085,
  2,
  1,
  27,
  67,
  21,
  12,
  3,
  7,
  26,
  13,
  8,
  23,
  14,
  30,
  46,
  39,
  102],
 [2836,
  4692,
  163,
  16,
  1047,
  45,
  2836,
  9,
  18,
  85,
  45,
  148,
  4693,
  376,
  2727,
  109,
  248,
  137,
  16,
  579,
  45,
  100,
  1076,
  51,
  19,
  1205,
  609,
  11,
  32,
  92,
  1

  1,
  25,
  1,
  25,
  198,
  227,
  44,
  5,
  42,
  40,
  10,
  39,
  362,
  363,
  87,
  356,
  5,
  107,
  330,
  74,
  362,
  11,
  10,
  29,
  31,
  53,
  398],
 [651,
  652,
  1602,
  2046,
  594,
  95,
  58,
  817,
  95,
  801,
  802,
  1602,
  255,
  424,
  623,
  480,
  2046,
  2,
  1,
  27,
  3,
  12,
  7,
  26,
  13,
  23,
  8],
 [2829,
  2830,
  2831,
  826,
  244,
  309,
  438,
  15,
  352,
  89,
  198,
  227,
  2832,
  3,
  5,
  558,
  244,
  309,
  708,
  438,
  15,
  1083,
  2833,
  156,
  276,
  106,
  1100,
  2834,
  50,
  4,
  2,
  1,
  93,
  97,
  1,
  383,
  147],
 [171,
  140,
  4752,
  2842,
  3,
  86,
  543,
  4753,
  229,
  297,
  4754,
  815,
  86,
  56,
  2842,
  3,
  289,
  1648,
  2494,
  610,
  61,
  4755,
  215,
  296,
  2531,
  654,
  1282,
  4756,
  310,
  544,
  86,
  68,
  922,
  747,
  15,
  1111,
  518,
  404,
  127,
  1037,
  256,
  92,
  1570,
  610,
  214,
  1272,
  1501,
  1225,
  86,
  326,
  4757,
  894,
  418,
  35,
  490,
  213,
  816,
  6

In [27]:
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

In [28]:
# get the max length in terms of token length
max_length = get_max_token_length_per_doc(docs)

In [29]:
max_length

150

In [30]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_docs)

Padded docs: [[  60   76  679 ...    0    0    0]
 [  56   49 1434 ...    0    0    0]
 [  56  124  303 ...    0    0    0]
 ...
 [  56  124 1683 ...    0    0    0]
 [  48 4796   37 ...    0    0    0]
 [ 216  486   19 ...    0    0    0]]


In [31]:
EMBEDDING_SIZE = 100

### Define deep learning model

In [32]:
# define the model

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
model.add(Flatten()) 

model.add(Dense(1, activation='sigmoid')) 

In [33]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          500000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 15001     
Total params: 515,001
Trainable params: 515,001
Non-trainable params: 0
_________________________________________________________________


In [34]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.834573


In [35]:
embedding_layer = model.layers[0]
embedding_layer.get_weights()[0].shape

(5000, 100)

In [36]:
test_docs = list(X_test)
test_labels = y_test

In [37]:
encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

# pad test documents
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_test_docs)

Padded docs: [[ 177  173   28 ...    0    0    0]
 [ 911  176   43 ...    0    0    0]
 [ 177  173   95 ...    0    0    0]
 ...
 [ 656 2998  244 ...    0    0    0]
 [ 264  103   19 ...    0    0    0]
 [ 494  953  762 ...    0    0    0]]


In [38]:
loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 96.774191


In [39]:
pattern.iloc[:, 29]

0       hunting season leather-paneled raffia shoulder...
1       hunting season leather-paneled raffia shoulder...
2       alexis calixte pussy-bow chiffon top alexis 'c...
3       derek lam belted lace-trimmed printed silk-cre...
4       mother pearl name black white lyocell partiall...
                              ...                        
1607    ulla johnson natalia metallic shibori long sle...
1608    zadig voltaire paros crystal buckle ankle stra...
1609    zadig voltaire used sneaker graffiti graphic c...
1610    zadig voltaire rock star circus metallic leath...
1611    sarto franco sarto bordo slide sandal blunted ...
Name: input_doc, Length: 1612, dtype: object

### For loop to test all attribute values based on the deep learning model

In [40]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [41]:
def get_max_token_length_per_doc(docs):
    return max(list(map(lambda x: len(x.split()), docs)))

In [42]:
X = pattern.iloc[:, -1].values
Accuracy = []
pattern_values = list(pattern.columns.values[9:29])
max_length_list = []
for i in range(9, 29):
    y = pattern.iloc[:, i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000
    EMBEDDING_SIZE = 100

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)
    
    #save model
    model.save("{}_model.h5".format(pattern_values[i-9]))

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    test_labels = y_test

    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

    loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0) # the average accuracy is around 95%
    #save tokenizer
    with open('{}_tokenizer.pickle'.format(pattern_values[i-9]), 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    max_length_list.append(max_length)
    Accuracy.append(accuracy)

In [43]:
max_length_list

[150,
 150,
 124,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150]

In [58]:
Accuracy

[0.9677419066429138,
 0.8982630372047424,
 0.9900744557380676,
 0.985111653804779,
 0.9727047085762024,
 0.9528536200523376,
 0.9429280161857605,
 0.9801489114761353,
 0.9950371980667114,
 0.9528536200523376,
 0.9975185990333557,
 0.9925558567047119,
 0.9925558567047119,
 0.9900744557380676,
 0.9503722190856934,
 0.9826302528381348,
 0.9751861095428467,
 0.9354838728904724,
 0.9826302528381348,
 0.9900744557380676]

In [44]:
d = {'pattern_value': pattern_values, 'accuracy': Accuracy}
accuracy_table = pd.DataFrame(data = d)
accuracy_table

pattern_value  accuracy
0           abstract  0.967742
1             animal  0.898263
2         camouflage  0.990074
3         colorblock  0.985112
4               dots  0.972705
5             floral  0.952854
6          geometric  0.942928
7            graphic  0.980149
8        houndstooth  0.995037
9               logo  0.952854
10          monogram  0.997519
11        multiprint  0.992556
12           paisley  0.992556
13         pinstripe  0.990074
14             plaid  0.950372
15            stripe  0.982630
16  stripehorizontal  0.975186
17    stripevertical  0.935484
18            tiedye  0.982630
19          tropical  0.990074

# Apply our trained model on full data

In [45]:
full_data = pd.read_csv('full_data_final version.csv')

In [46]:
#replace null values with UNKNOWN_TOKEN
full_data['description'] = full_data['description'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].fillna('UNKNOWN_TOKEN')
full_data['brand_category'] = full_data['brand_category'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].str.replace("\n", "")

In [47]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    full_data[col] = full_data[col].apply(removePunctuation)
    full_data[col] = full_data[col].apply(removeStopwords)
    full_data[col] = full_data[col].apply(lemmatize)

In [48]:
full_data['abstract'] = 0
full_data['animal'] = 0
full_data['camouflage'] = 0
full_data['colorblock'] = 0
full_data['dots'] = 0
full_data['floral'] = 0
full_data['geometric'] = 0
full_data['graphic'] = 0
full_data['houndstooth'] = 0
full_data['logo'] = 0
full_data['monogram'] = 0
full_data['multiprint'] = 0
full_data['paisley'] = 0
full_data['pinstripe'] = 0
full_data['plaid'] = 0
full_data['stripe'] = 0
full_data['stripehorizontal'] = 0
full_data['stripevertical'] = 0
full_data['tiedye'] = 0
full_data['tropical'] = 0
full_data["input_doc"] = full_data.brand+" "+full_data.product_full_name+" "+full_data.description+" "+full_data.details 

In [49]:
full_data.head()

product_id              brand          mpn  \
0  01DSE9TC2DQXDG6GWKW9NMJ416    banana republic       514683   
1  01DSE9SKM19XNA6SJP36JZC065    banana republic       526676   
2  01DSJX8GD4DSAP76SPR85HRCMN              loewe    4.001E+11   
3  01DSJVKJNS6F4KQ1QM6YYK9AW2           converse  4.00012E+11   
4  01DSK15ZD4D5A0QXA8NSD25YXE  alexander mcqueen  4.00011E+11   

                                   product_full_name  \
0                                   ankle-strap pump   
1                                petite tie-neck top   
2                 52mm padded leather round sunglass   
3  baby 's little kid 's all-star two-tone mid-to...   
4                              64mm rimless sunglass   

                                         description  \
0  modern pump rounded silhouette ankle strap ext...   
1  dress jean sneaker dress tailored trouser heel...   
2        padded leather cover classic round sunglass   
3  iconic mid-top design get added dose support p...   
4  hexagonal shade offer rimless view intricate n...   

                                      brand_category  \
0                                            Unknown   
1                                            Unknown   
2  JewelryAccessories/SunglassesReaders/RoundOval...   
3  JustKids/Shoes/Baby024Months/BabyGirl,JustKids...   
4     JewelryAccessories/SunglassesReaders/RoundOval   

                      created_at                     updated_at deleted_at  \
0  2019-11-11 22:37:15.719107+00  2019-12-19 20:40:30.786144+00        NaN   
1  2019-11-11 22:36:50.682513+00  2019-12-19 20:40:30.786144+00        NaN   
2  2019-11-13 17:33:59.581661+00  2019-12-19 20:40:30.786144+00        NaN   
3  2019-11-13 17:05:05.203733+00  2019-12-19 20:40:30.786144+00        NaN   
4  2019-11-13 18:42:30.941321+00  2019-12-19 20:40:30.786144+00        NaN   

                                 brand_canonical_url  ... multiprint paisley  \
0  https://bananarepublic.gap.com/browse/product....  ...          0       0   
1  https://bananarepublic.gap.com/browse/product....  ...          0       0   
2  https://www.saksfifthavenue.com/loewe-52mm-pad...  ...          0       0   
3  https://www.saksfifthavenue.com/converse-babys...  ...          0       0   
4  https://www.saksfifthavenue.com/alexander-mcqu...  ...          0       0   

   pinstripe  plaid  stripe  stripehorizontal  stripevertical  tiedye  \
0          0      0       0                 0               0       0   
1          0      0       0                 0               0       0   
2          0      0       0                 0               0       0   
3          0      0       0                 0               0       0   
4          0      0       0                 0               0       0   

   tropical                                          input_doc  
0         0  banana republic ankle-strap pump modern pump r...  
1         0  banana republic petite tie-neck top dress jean...  
2         0  loewe 52mm padded leather round sunglass padde...  
3         0  converse baby 's little kid 's all-star two-to...  
4         0  alexander mcqueen 64mm rimless sunglass hexago...  

[5 rows x 34 columns]

In [50]:
full_data.shape

(48979, 34)

In [51]:
for i in range(13, 33):
    X_train = pattern.iloc[:, -1].values
    y_train = pattern.iloc[:, i-4].values
    X_test = full_data.iloc[:, -1].values

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    
    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
    prediction = model.predict(padded_test_docs, verbose=0)
    full_data.iloc[:, i] = prediction

In [52]:
full_data.head()

product_id              brand          mpn  \
0  01DSE9TC2DQXDG6GWKW9NMJ416    banana republic       514683   
1  01DSE9SKM19XNA6SJP36JZC065    banana republic       526676   
2  01DSJX8GD4DSAP76SPR85HRCMN              loewe    4.001E+11   
3  01DSJVKJNS6F4KQ1QM6YYK9AW2           converse  4.00012E+11   
4  01DSK15ZD4D5A0QXA8NSD25YXE  alexander mcqueen  4.00011E+11   

                                   product_full_name  \
0                                   ankle-strap pump   
1                                petite tie-neck top   
2                 52mm padded leather round sunglass   
3  baby 's little kid 's all-star two-tone mid-to...   
4                              64mm rimless sunglass   

                                         description  \
0  modern pump rounded silhouette ankle strap ext...   
1  dress jean sneaker dress tailored trouser heel...   
2        padded leather cover classic round sunglass   
3  iconic mid-top design get added dose support p...   
4  hexagonal shade offer rimless view intricate n...   

                                      brand_category  \
0                                            Unknown   
1                                            Unknown   
2  JewelryAccessories/SunglassesReaders/RoundOval...   
3  JustKids/Shoes/Baby024Months/BabyGirl,JustKids...   
4     JewelryAccessories/SunglassesReaders/RoundOval   

                      created_at                     updated_at deleted_at  \
0  2019-11-11 22:37:15.719107+00  2019-12-19 20:40:30.786144+00        NaN   
1  2019-11-11 22:36:50.682513+00  2019-12-19 20:40:30.786144+00        NaN   
2  2019-11-13 17:33:59.581661+00  2019-12-19 20:40:30.786144+00        NaN   
3  2019-11-13 17:05:05.203733+00  2019-12-19 20:40:30.786144+00        NaN   
4  2019-11-13 18:42:30.941321+00  2019-12-19 20:40:30.786144+00        NaN   

                                 brand_canonical_url  ... multiprint  \
0  https://bananarepublic.gap.com/browse/product....  ...   0.000089   
1  https://bananarepublic.gap.com/browse/product....  ...   0.000453   
2  https://www.saksfifthavenue.com/loewe-52mm-pad...  ...   0.000629   
3  https://www.saksfifthavenue.com/converse-babys...  ...   0.000489   
4  https://www.saksfifthavenue.com/alexander-mcqu...  ...   0.000146   

    paisley  pinstripe     plaid    stripe  stripehorizontal  stripevertical  \
0  0.000184   0.000228  0.000084  0.000200          0.000031        0.001168   
1  0.000127   0.000090  0.000023  0.000594          0.000346        0.000210   
2  0.000062   0.000946  0.001584  0.000715          0.000628        0.022307   
3  0.000463   0.000458  0.000102  0.001631          0.000894        0.001470   
4  0.000127   0.000633  0.004992  0.000620          0.000539        0.002874   

     tiedye  tropical                                          input_doc  
0  0.000024  0.000116  banana republic ankle-strap pump modern pump r...  
1  0.000177  0.000269  banana republic petite tie-neck top dress jean...  
2  0.000337  0.001105  loewe 52mm padded leather round sunglass padde...  
3  0.000207  0.004406  converse baby 's little kid 's all-star two-to...  
4  0.000460  0.000974  alexander mcqueen 64mm rimless sunglass hexago...  

[5 rows x 34 columns]

In [53]:
df1 = full_data.iloc[:, 13:33]
df1.head()

abstract    animal  camouflage  colorblock      dots    floral  geometric  \
0  0.000091  0.917049    0.000121    0.000269  0.004221  0.001241   0.000294   
1  0.000219  0.010646    0.000048    0.000004  0.002208  0.023376   0.013775   
2  0.005466  0.045991    0.000204    0.001203  0.004327  0.000438   0.009836   
3  0.006016  0.002848    0.000089    0.000321  0.002011  0.004377   0.008075   
4  0.002699  0.029886    0.000259    0.001942  0.001880  0.000452   0.003260   

        graphic  houndstooth      logo  monogram  multiprint   paisley  \
0  2.575489e-05     0.000021  0.002455  0.000366    0.000089  0.000184   
1  5.983576e-07     0.000005  0.000026  0.000076    0.000453  0.000127   
2  3.723569e-03     0.000092  0.176381  0.000169    0.000629  0.000062   
3  3.718206e-04     0.000094  0.039098  0.000235    0.000489  0.000463   
4  1.494953e-03     0.000048  0.088017  0.000160    0.000146  0.000127   

   pinstripe     plaid    stripe  stripehorizontal  stripevertical    tiedye  \
0   0.000228  0.000084  0.000200          0.000031        0.001168  0.000024   
1   0.000090  0.000023  0.000594          0.000346        0.000210  0.000177   
2   0.000946  0.001584  0.000715          0.000628        0.022307  0.000337   
3   0.000458  0.000102  0.001631          0.000894        0.001470  0.000207   
4   0.000633  0.004992  0.000620          0.000539        0.002874  0.000460   

   tropical  
0  0.000116  
1  0.000269  
2  0.001105  
3  0.004406  
4  0.000974

In [54]:
df1['patterns/prints'] = df1.idxmax(axis = 1)

In [56]:
full_data['patterns/prints'] = df1['patterns/prints']

In [57]:
full_data.head()

product_id              brand          mpn  \
0  01DSE9TC2DQXDG6GWKW9NMJ416    banana republic       514683   
1  01DSE9SKM19XNA6SJP36JZC065    banana republic       526676   
2  01DSJX8GD4DSAP76SPR85HRCMN              loewe    4.001E+11   
3  01DSJVKJNS6F4KQ1QM6YYK9AW2           converse  4.00012E+11   
4  01DSK15ZD4D5A0QXA8NSD25YXE  alexander mcqueen  4.00011E+11   

                                   product_full_name  \
0                                   ankle-strap pump   
1                                petite tie-neck top   
2                 52mm padded leather round sunglass   
3  baby 's little kid 's all-star two-tone mid-to...   
4                              64mm rimless sunglass   

                                         description  \
0  modern pump rounded silhouette ankle strap ext...   
1  dress jean sneaker dress tailored trouser heel...   
2        padded leather cover classic round sunglass   
3  iconic mid-top design get added dose support p...   
4  hexagonal shade offer rimless view intricate n...   

                                      brand_category  \
0                                            Unknown   
1                                            Unknown   
2  JewelryAccessories/SunglassesReaders/RoundOval...   
3  JustKids/Shoes/Baby024Months/BabyGirl,JustKids...   
4     JewelryAccessories/SunglassesReaders/RoundOval   

                      created_at                     updated_at deleted_at  \
0  2019-11-11 22:37:15.719107+00  2019-12-19 20:40:30.786144+00        NaN   
1  2019-11-11 22:36:50.682513+00  2019-12-19 20:40:30.786144+00        NaN   
2  2019-11-13 17:33:59.581661+00  2019-12-19 20:40:30.786144+00        NaN   
3  2019-11-13 17:05:05.203733+00  2019-12-19 20:40:30.786144+00        NaN   
4  2019-11-13 18:42:30.941321+00  2019-12-19 20:40:30.786144+00        NaN   

                                 brand_canonical_url  ...   paisley pinstripe  \
0  https://bananarepublic.gap.com/browse/product....  ...  0.000184  0.000228   
1  https://bananarepublic.gap.com/browse/product....  ...  0.000127  0.000090   
2  https://www.saksfifthavenue.com/loewe-52mm-pad...  ...  0.000062  0.000946   
3  https://www.saksfifthavenue.com/converse-babys...  ...  0.000463  0.000458   
4  https://www.saksfifthavenue.com/alexander-mcqu...  ...  0.000127  0.000633   

      plaid    stripe  stripehorizontal  stripevertical    tiedye  tropical  \
0  0.000084  0.000200          0.000031        0.001168  0.000024  0.000116   
1  0.000023  0.000594          0.000346        0.000210  0.000177  0.000269   
2  0.001584  0.000715          0.000628        0.022307  0.000337  0.001105   
3  0.000102  0.001631          0.000894        0.001470  0.000207  0.004406   
4  0.004992  0.000620          0.000539        0.002874  0.000460  0.000974   

                                           input_doc  patterns/prints  
0  banana republic ankle-strap pump modern pump r...           animal  
1  banana republic petite tie-neck top dress jean...           floral  
2  loewe 52mm padded leather round sunglass padde...             logo  
3  converse baby 's little kid 's all-star two-to...             logo  
4  alexander mcqueen 64mm rimless sunglass hexago...             logo  

[5 rows x 35 columns]